In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=junn password=junn")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute("DROP DATABASE accounts;")
    cur.execute("CREATE DATABASE accounts;")
    conn.close()

    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=junn password=junn")
    cur = conn.cursor()
    return conn, cur

In [29]:
conn, cur = create_database()

ObjectInUse: database "accounts" is being accessed by other users
DETAIL:  There is 1 other session using the database.


In [10]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [11]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [12]:
AccountsCountry = pd.read_csv("data/Wealth-AccountsCountry.csv")

In [13]:
AccountsCountry_clean = AccountsCountry[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [14]:
AccountsData = pd.read_csv('data/Wealth-AccountData.csv')
AccountsData = AccountsData[['Country Name', 'Country Code', 'Series Name', 'Series Code', '1995 [YR1995]', 
              '2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]', '2014 [YR2014]']]

In [15]:
AccountsCountry_clean = AccountsCountry_clean.rename(columns={'Code': 'Country Code'})

In [16]:
AccountSeries = pd.read_csv('data/Wealth-AccountSeries.csv')

In [17]:
AccountSeries = AccountSeries.rename(columns={'Code': 'Series Code'})

In [18]:
AccountSeries = AccountSeries[['Series Code', 'Topic', 'Indicator Name', 'Long definition']]

In [19]:
table_create_query = ("""CREATE TABLE IF NOT EXISTS accountscountry(
country_code VARCHAR PRIMARY KEY,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
)""")

accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR,
year_1995 NUMERIC,
year_2000 NUMERIC,
year_2005 NUMERIC,
year_2010 NUMERIC,
year_2014 NUMERIC
)""")

accounts_series_table_create = ("""CREATE TABLE IF NOT EXISTS accountsseries(
series_code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR,
long_definition VARCHAR
)""")

In [20]:
cur.execute(table_create_query)
conn.commit()

cur.execute(accounts_data_table_create)
conn.commit()

cur.execute(accounts_series_table_create)
conn.commit()

In [21]:
accounts_country_table_insert = ("""INSERT INTO accountscountry(
country_code,
short_name,
table_name,
long_name,
currency_unit)
VALUES (%s, %s, %s, %s, %s)""")

In [27]:
for i, row in AccountsCountry_clean.iterrows():
    cur.execute(accounts_country_table_insert, list(row))

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [26]:
AccountsCountry_clean = AccountsCountry_clean.dropna()